# Where Do We Need More Electric Vehicle Charging Stations? Visualizing Supply Demand by Zip Codes

The switch to electric vehicles (EVs) requires adequate public charging infrastructure. Which areas are most in need? This project shows the geographic areas (zip codes) where the ratio of available public charging stations to registered EVs is the least, areas most in need of additional charging infrastructure.

In [ ]:
import pandas as pd
import numpy as np
from functools import reduce
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")


## Getting data

Several data sources were combined for this project using zip code as the primary key. The year 2016 is used due to data availability to provide uniform measures for comparisons. The American Community Survey (ACS) data by U.S.Census Bureau was the primary source of demographic and economic data. The location of EV charging stations was extracted from U.S. Department of Energy dataset. We are able to gather EV registration data for some states through Atlas EV HUB. Some states report EV registration by county, rather than by zip code, which we transformed using a cross-reference table from https://www.kaggle.com/datasets/danofer/zipcodes-county-fips-crosswalk which leverages US HUD and Census Bureau data sources.

In [ ]:
data = pd.read_csv("Data/ACS data/master_data.csv")
data.describe()

## K-means clustering

Clustering of the demographic data is used to group zip codes with similar predictors. The clusters show high, middle, and low adoption rates. By grouping zip codes we can use estimate EV registration for zip codes that do not have EV registration data.

First the data will be cleaned and missing data will be imputed.

In [ ]:
# Replace all NaN with zeros (NO LONGER NEEDED?????)
data = data.apply(lambda x: x.replace("NaN", np.nan))
data = data.apply(lambda x: x.replace("-", np.nan))
data = data.apply(lambda x: x.replace("(X)", "out"))
data = data.drop(columns=data.columns[(data == "out").any()])

# fill in 0 for only the NUM_OF_EVS column, not rest of the dataframe??
data.fillna(0, inplace=True)

# Retrieve data where the target column has no values
data_without_reg = data.loc[(data["NUM_OF_EVS"] == 0)]

# Retrieve data where the target column has values
data_with_reg = data.loc[(data["NUM_OF_EVS"] != 0)]

# Declare feature vector and target variable
X = data_with_reg

# Replace commas and other special characters (NO LONGER NEEDED???)
X = (
    X.replace(",", "", regex=True)
    .replace("-", "", regex=True)
    .replace("\+", "", regex=True)
)

# Remove ZIP, State, etc from dataframe
X = X.iloc[:, 5:]
X = X.astype((float))
y = data_with_reg["NUM_OF_EVS"]

# Impute missing data for all columns with the median of each column
X = X.apply(lambda x: x.fillna(x.median(), axis=0))

### Principal Component Analysis

Principal Component Analysis (PCA) is used on datasets with a high number of features to reduce dimensionality while maintaining information.

In [ ]:
# Feature Scaling
cols = X.columns
ms = MinMaxScaler()
t = ms.fit_transform(X)
X = pd.DataFrame(t, columns=[cols])

pcaMod = PCA()
X = pcaMod.fit_transform(X)

### Elbow Plot

An elbow plot is used to determine the optimal value of clusters. The point where the gain stops increasing is the optimal k value or number of clusters.

In [ ]:
distortions = []
K = range(1, 10)
for k in K:
    kmeanModel = KMeans(n_clusters=k)
    kmeanModel.fit(X)
    distortions.append(kmeanModel.inertia_)

plt.figure(figsize=(16, 8))
plt.plot(K, distortions, "bx-")
plt.xlabel("k")
plt.ylabel("Distortion")
plt.title("The Elbow Method showing the optimal k")
plt.show()

### K-means clustering with 3 clusters

After determining the optimal number of clusters as 3 the k-means clustering algorithm will be run again.

In [ ]:
# Initialize kmeans object
kmeans = KMeans(n_clusters=3, random_state=0)

# Predict the labels of clusters
label = kmeans.fit_predict(X)

# Getting the Centroids
centroids = kmeans.cluster_centers_

# Unique labels
uniqueLabels = np.unique(label)
print("Unique Kmeans Labels === ", uniqueLabels)

for i in uniqueLabels:
    plt.scatter(X[label == i, 0], X[label == i, 1], label=i, s=30)
plt.scatter(centroids[:, 0], centroids[:, 1], s=100, color="black")
plt.legend()
plt.show()

### Based on clustering groups, filter the original data using the labels

Now that the data is separated into 3 clusters the original data will be labeled.

In [ ]:
# Filter rows of the oringinal data frame - Cluster 0
filteredLabel0 = data_with_reg[label == 0]

# Add column ClusterLabel and set to zero for this group
filteredLabel0.loc[:, "ClusterLabel"] = 0

# Filter rows of the oringinal data frame - Cluster 1
filteredLabel1 = data_with_reg[label == 1]

# Add column ClusterLabel and set to one for this group
filteredLabel1.loc[:, "ClusterLabel"] = 1

# Filter rows of the oringinal data frame - Cluster 2
filteredLabel2 = data_with_reg[label == 2]

# Add column ClusterLabel and set to zero for this group
filteredLabel2.loc[:, "ClusterLabel"] = 2

cluster_df = pd.concat(
    [filteredLabel0, filteredLabel1, filteredLabel2], ignore_index=0, axis=0
)

### K nearest neighbors

A K Nearest Neighbors model is used to classify each zip code that did not have EV registration data.

In [ ]:
knnMod = KNeighborsClassifier(n_neighbors=15)
X1 = cluster_df.iloc[:, 5:543]
# Replace commas and other special characters (NO LONGER NEEDED)
X1 = (
    X1.replace(",", "", regex=True)
    .replace("-", "", regex=True)
    .replace("\+", "", regex=True)
    .astype((float))
)
Y1 = cluster_df.iloc[:, 544]

# Build Model
method2_fit = knnMod.fit(X1, Y1)
print("Fit Score: ", method2_fit.score(X1, Y1))

X1_test = data_without_reg
X1_test = X1_test.iloc[:, 5:543]
# Replace commas and other special characters (NO LONGER NEEDED)
X1_test = (
    X1_test.replace(",", "", regex=True)
    .replace("-", "", regex=True)
    .replace("\+", "", regex=True)
    .astype((float))
)

knnLabels = method2_fit.predict(X1_test)
print("Unique KNN Labels ==", np.unique(knnLabels))

# Based on Classification groups, filter the original data without registration using the labels
# Filter rows of the oringinal data frame - Cluster 0
knnLabel0 = data_without_reg[knnLabels == 0]
knnLabel0.loc[:, "ClusterLabel"] = 0

knnLabel1 = data_without_reg[knnLabels == 1]
knnLabel1.loc[:, "ClusterLabel"] = 1

knnLabel2 = data_without_reg[knnLabels == 2]
knnLabel2.loc[:, "ClusterLabel"] = 2

classification_df = pd.concat([knnLabel0, knnLabel1, knnLabel2], ignore_index=0, axis=0)


### Linear Regression

We will use the OLS regression model since Herling 2019 has shown an effective way to model EV  adoption. This paper also provides significant factors of the OLS regression model that affect the rate of EV adoption by zip code in Massachusetts. We can use a similar method for the rest of the country. Another study by Vergis et al.xvii describes potential explanatory variables for EV adoption, including existing charging stations and gas prices. 

Since we know the actual EV registration for some zip codes we used this dataset to train a regression model and impute EV registrations for zip codes without the this information with predicted values. The steps for building the regression models are outlines below.

 - Start with clustered data with registration information available
 - For each cluster (0,1,2), perform features selection to identify top 10 best scoring predictors using Univariate feature selection
     - Start with over 500 predictors which included granular demographic data (Education, Age, income, transportation type, gender, household characteristics, etc) along with number of charging stations, gas prices, electricity rates.
     - We want to build a regression model with smaller number of features and avoid overfitting.

The features selected for each cluster align with the characteristics of each cluster as stated. For cluster 0 with more suburbs the EV registration was correlated to education, home value, mortgage (proxy for income) and transportation. For cluster 1, with more rural areas the EV registration was correlated with level of income, unemployment, household characteristics, transportation, and rent. For cluster 2, witmore urban areas the correlation was with home values, mortgage (proxy for income) and transportation type.

Once three regression models were created for each cluster label, the same models were used to predict EV registration for zip codes were the data was not available. Depending on the classification label (0,1,2), the corresponding regression model was used to yield a predicted EV registration. The predictions were clipped at a minimum of 0 and values were rounded to nearest integer. 

A few observations on the predictions:
- Areas with high number of charging stations show higher EV registration
- Areas with no charging station tended to have 0 for EV registrations
- Top 10 states with highest EV reg were : CA, NY, NJ, TX, FL, PA, IL, MA, MN, and OR
- Top 10 States with most charging station: CA, NY, FL, TX, MA, WA, CO, GA, MD, VA
- This indicates some states like NJ, IL, MN, and OR might have high demand for charging stations
- Top 10 States with lowest EV reg: DE, WY, RI, AK, HI, ID, VT, ND, DC, SD
- Top 10 states with lowest # of chargers: AK, SD, ND, WY, MT, WV, MS, ID, DE, LA
- These are some of the rural or smaller states. This implies some states have larger counts perhaps because of their geographic size.
- If we focus on zipcodes, following states are still on top of the list: NJ, OR, VA, CA, WA, OR, MI, TX
- Similarly, AL, AR, AZ are in lower EV Reg list based on zipcodes

In [ ]:
all_features = cluster_df.iloc[:, 5:541].columns.tolist()
all_features.append("NUM_OF_CHARGERS")

# feature selection for cluster 0
cluster_0 = cluster_df.loc[(cluster_df["ClusterLabel"] == 0)]
X0 = cluster_0[all_features]
Y0 = cluster_0["NUM_OF_EVS"]

# Univariate feature Selection
skbest0 = SelectKBest(score_func=f_classif, k=10)
skbest0_fit = skbest0.fit(X0, Y0)
X_0 = skbest0_fit.transform(X0)

# split available data to test/train for cluster 0
# X_0 = cluster_0[features]
Y_0 = cluster_0["NUM_OF_EVS"]
X0_train, X0_test, y0_train, y0_test = train_test_split(
    X_0, Y_0, test_size=0.30, random_state=35
)

# Run regression 0 and check metrics
reg0 = LinearRegression().fit(X0_train, y0_train)
y0_pred_train = reg0.predict(X0_train).astype(int)
y0_pred_test = reg0.predict(X0_test).astype(int)

print("MSE of training data (0):", mean_squared_error(y0_train, y0_pred_train))
print("MSE(0):", mean_squared_error(y0_test, y0_pred_test))
print("r^2 of training data (0):", r2_score(y0_train, y0_pred_train))
print("r^2(0):", r2_score(y0_test, y0_pred_test))

############ feature selection for cluster 1  #################
cluster_1 = cluster_df.loc[(cluster_df["ClusterLabel"] == 1)]
X1 = cluster_1[all_features]
Y1 = cluster_1["NUM_OF_EVS"]

# Univariate feature Selection
skbest1 = SelectKBest(score_func=f_classif, k=10)
skbest1_fit = skbest1.fit(X1, Y1)
X_1 = skbest1_fit.transform(X1)

Y_1 = cluster_1["NUM_OF_EVS"]
X1_train, X1_test, y1_train, y1_test = train_test_split(
    X_1, Y_1, test_size=0.30, random_state=35
)

# Run regression 2 and check metrics
reg1 = LinearRegression().fit(X1_train, y1_train)
y1_pred_train = reg1.predict(X1_train).astype(int)
y1_pred_test = reg1.predict(X1_test).astype(int)

print("MSE of training data (1):", mean_squared_error(y0_train, y0_pred_train))
print("MSE(1):", mean_squared_error(y1_test, y1_pred_test))
print("r^2 of training data (1):", r2_score(y0_train, y0_pred_train))
print("r^2(1):", r2_score(y1_test, y1_pred_test))

############ feature selection for cluster 2  #################
cluster_2 = cluster_df.loc[(cluster_df["ClusterLabel"] == 2)]
X2 = cluster_2[all_features]
Y2 = cluster_2["NUM_OF_EVS"]

# Univariate feature Selection
skbest2 = SelectKBest(score_func=f_classif, k=10)
skbest2_fit = skbest2.fit(X2, Y2)
X_2 = skbest2_fit.transform(X2)

Y_2 = cluster_2["NUM_OF_EVS"]
X2_train, X2_test, y2_train, y2_test = train_test_split(
    X_2, Y_2, test_size=0.30, random_state=35
)

# Run regression 2 and check metrics
reg2 = LinearRegression().fit(X2_train, y2_train)
y2_pred_train = reg2.predict(X2_train).astype(int)
y2_pred_test = reg2.predict(X2_test).astype(int)

print("MSE of training data (2):", mean_squared_error(y0_train, y0_pred_train))
print("MSE(2):", mean_squared_error(y2_test, y2_pred_test))
print("r^2 of training data (2):", r2_score(y0_train, y0_pred_train))
print("r^2(2):", r2_score(y2_test, y2_pred_test))

# Predict Num_of_EVs for rest of the zipcodes using regression model built
# Summary of data with registration by clusters
class_0 = classification_df.loc[(classification_df["ClusterLabel"] == 0)]
XC0 = class_0[all_features]
YC0 = class_0["NUM_OF_EVS"]
XC_0 = skbest0_fit.transform(XC0)

class_1 = classification_df.loc[(classification_df["ClusterLabel"] == 1)]
XC1 = class_1[all_features]
YC1 = class_1["NUM_OF_EVS"]
XC_1 = skbest1_fit.transform(XC1)

class_2 = classification_df.loc[(classification_df["ClusterLabel"] == 2)]
XC2 = class_2[all_features]
YC2 = class_2["NUM_OF_EVS"]

XC_2 = skbest2_fit.transform(XC2)

Num_of_EVS_pred0 = reg0.predict(XC_0)
class_0["NUM_OF_EVS"] = Num_of_EVS_pred0.astype(int)
class_0["NUM_OF_EVS"] = np.where(class_0["NUM_OF_EVS"] < 0, 0, class_0["NUM_OF_EVS"])

Num_of_EVS_pred1 = reg1.predict(XC_1)
class_1["NUM_OF_EVS"] = Num_of_EVS_pred1.astype(int)
class_1["NUM_OF_EVS"] = np.where(class_1["NUM_OF_EVS"] < 0, 0, class_1["NUM_OF_EVS"])

Num_of_EVS_pred2 = reg2.predict(XC_2)
class_2["NUM_OF_EVS"] = Num_of_EVS_pred2.astype(int)
class_2["NUM_OF_EVS"] = np.where(class_2["NUM_OF_EVS"] < 0, 0, class_2["NUM_OF_EVS"])

# combine all data into one df
combined_df = pd.concat(
    [class_0, class_1, class_2, cluster_0, cluster_1, cluster_2], axis=0
)
regression_df = combined_df[["ZIP", "NUM_OF_EVS", "NUM_OF_CHARGERS"]]
regression_df["EV_CHARGER_RATIO"] = (
    regression_df["NUM_OF_EVS"] / regression_df["NUM_OF_CHARGERS"]
)

# Write out csv file for visualization
# regression_df.to_csv("Data/Final_data.csv", index=False)

regression_df.head()

## Interactive Map with a ratio of Electrical Vehicle registrations per chargin station for each US zip code.

States with Electrical Vehicle (EV) registration data is used to determine the EV registration for states without registration data. Using the number of EV registrations for each zip code and the current number of charging stations in each zip code a ratio of EVs to chargers is calculated. The EV charger to charging station ratio is visualized on a US Map.

### Merge State Geometry with State abbreviation

A shape file of US zip codes is used to represent each zip code. The shape file geopandas dataframe is merged with a zip code to state translation file inorder to add the state id.

In [ ]:
# Load file with state geometry
zip_data = gpd.read_file(
    r"Data\shapefile2018\cb_2018_us_zcta510_500k.shp"
)  # need folder of all shape files together but just need to point to this file.
zip_data["ZCTA5CE10"] = zip_data["ZCTA5CE10"].astype(
    "int"
)  # converting zipcode to int for merge with charger data

# Add State to zip code mapping
zip2state = pd.read_csv(r"data\zip2state.csv", usecols=["zip", "state_id"])

# Merge state with shapefile
zip_data = zip_data.merge(zip2state, left_on="ZCTA5CE10", right_on="zip")
zip_data = zip_data.drop(["zip"], axis=1)


## Charging Station data

Charging Station data is obtained from https://afdc.energy.gov/fuels/electricity_locations.html#/find/nearest?fuel=ELEC. The charging station data is used to display the charging station details as a tool tip

In [ ]:
# Load file with EV Charger data
chargers = pd.read_csv(
    r"Data\alt_fuel_stations (Mar 22 2022).csv",
    usecols=[
        "State",
        "ZIP",
        "EV Level1 EVSE Num",
        "EV Level2 EVSE Num",
        "EV DC Fast Count",
    ],
)
chargers = chargers.pivot_table(
    chargers,
    index=["ZIP", "State"],
    aggfunc={
        "EV Level1 EVSE Num": np.sum,
        "EV Level2 EVSE Num": np.sum,
        "EV DC Fast Count": np.sum,
    },
).reset_index()

# Merge datasets
us_states = zip_data.merge(chargers, left_on="ZCTA5CE10", right_on="ZIP", how="left")
us_states = us_states.drop(["ZIP", "State"], axis=1)
us_states.shape

# Replace NaN with 0
# us_states = us_states.loc[~us_states['state_id'].isin(['AK', 'HI', 'PR'])]
us_states["EV Level1 EVSE Num"] = us_states["EV Level1 EVSE Num"].fillna(0)
us_states["EV Level2 EVSE Num"] = us_states["EV Level2 EVSE Num"].fillna(0)
us_states["EV DC Fast Count"] = us_states["EV DC Fast Count"].fillna(0)
us_states = us_states.rename(
    columns={
        "EV Level1 EVSE Num": "EV_Level1_EVSE_Num",
        "EV Level2 EVSE Num": "EV_Level2_EVSE_Num",
        "EV DC Fast Count": "EV_DC_Fast_Count",
    }
)


## Add EV to charger ratio values to DataFrame

The calculated ratio values are merged to the DataFrame.


In [ ]:
us_states = us_states.merge(regression_df, left_on="ZCTA5CE10", right_on="ZIP")
us_states = us_states.drop(["ZIP"], axis=1)

# Remove zip codes that don't have any EV Registrations
# us_states = us_states[us_states["NUM_OF_EVS"] > 0]
us_states["NUM_OF_CHARGERS"] = (
    us_states["EV_DC_Fast_Count"]
    + us_states["EV_Level1_EVSE_Num"]
    + us_states["EV_Level2_EVSE_Num"]
)
us_states["EV_CHARGER_RATIO"] = us_states["NUM_OF_CHARGERS"] / us_states["NUM_OF_EVS"]
us_states["EV_CHARGER_RATIO"] = us_states["EV_CHARGER_RATIO"].fillna(0)
us_states["EV_CHARGER_RATIO"] = us_states["EV_CHARGER_RATIO"].replace(np.inf, 0)
us_states[us_states["NUM_OF_EVS"] == 0].head()

## Create Visualization with EV per Charging Station ratio

In [ ]:
# Import packages for building map
from bokeh.io import show, save, output_file
from bokeh.models import (
    ColorBar,
    GeoJSONDataSource,
    HoverTool,
    LinearColorMapper,
    Slider,
    CustomJS,
    CustomJSFilter,
    CDSView,
)
from bokeh.palettes import brewer
from bokeh.plotting import figure
from bokeh.layouts import column, widgetbox

# Input GeoJSON source that contains features for plotting
geosource = GeoJSONDataSource(geojson=us_states.to_json())

In [ ]:
# Define color palettes
output_file_path = r"ev_chargers.html"
palette = brewer["Spectral"][11]

# Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
low_value = us_states["EV_CHARGER_RATIO"].min()
high_value = 1  # us_states["EV_CHARGER_RATIO"].max()
color_mapper = LinearColorMapper(palette=palette, low=low_value, high=high_value)

# Define custom tick labels for color bar.
tick_labels = {
    "-1": "-1",
    "0.1": "0.1",
    "0.2": "0.2",
    "0.3": "0.3",
    "0.4": "0.4",
    "0.5": "0.5",
    "0.6": "0.6",
    "0.7": "0.7",
    "0.8": "0.8",
    "0.9": "0.9",
    "1": "1+",
}

# Create color bar.
color_bar = ColorBar(
    color_mapper=color_mapper,
    label_standoff=8,
    width=500,
    height=20,
    border_line_color=None,
    location=(0, 0),
    orientation="horizontal",
    major_label_overrides=tick_labels,
)

# Create figure object.
p = figure(
    title="Electrical Vehicle Chargers per Electrical Vehicle Registrations",
    plot_height=600,
    plot_width=950,
    toolbar_location="below",
    tools="pan, wheel_zoom, box_zoom, reset",
    x_range=(-140, -50),
    y_range=(22, 52),
)
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None

# Add patch renderer to figure.
zips = p.patches(
    "xs",
    "ys",
    source=geosource,
    fill_color={"field": "EV_CHARGER_RATIO", "transform": color_mapper},
    line_color="black",
    line_width=0.5,
    fill_alpha=1,
)

# Create hover tool
p.add_tools(
    HoverTool(
        renderers=[zips],
        tooltips=[
            ("Zipcode", "@ZCTA5CE10"),
            ("EV Charger Ratio", "@EV_CHARGER_RATIO"),
            ("Number of EV Registrations", "@NUM_OF_EVS"),
            ("Level 1", "@EV_Level1_EVSE_Num"),
            ("Level 2", "@EV_Level2_EVSE_Num"),
            ("DC Fast", "@EV_DC_Fast_Count"),
        ],
    )
)

# Specify layout
p.add_layout(color_bar, "below")
output_file(output_file_path)
save(p)
show(p)